<a href="https://colab.research.google.com/github/AlpinDale/misc-scripts/blob/main/Aphrodite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to the Aphrodite Engine Colab!

The default model is Mythalion-13B, but you can either type in your model name, or select one of the defaults in the dropdown. Please don't paste in the full HuggingFace URL, but only the Username/Modelname part. For example, `https://huggingface.co/TheBloke/Mythalion-13B-AWQ` would be `TheBloke/Mythalion-13B-AWQ` and so on.

If you're on mobile, please tap on the play button below. If you're not, you can safely skip it and go to the next cell.

NOTE: If you run into any problems, open an issue [here](https://github.com/AlpinDale/misc-scripts/issues).

NOTE: For **20B models**, use GPTQ and set `GPU_Memory_Utilization` to 0.9. Make sure you don't request more than 200 tokens per reply, as it may run out of memory.

In [ ]:
#@title <b>v-- Tap this if you play on Mobile</b> { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldAI below (You can ignore this step if you used run all and are on PC)</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- Run this cell to start the engine.</b>
#@markdown **The free plan on Google Colab only supports up to 13B (in GPTQ/AWQ) formats.**
#@markdown **You can enter a custom model as well (for models bigger than 13B GPTQ/AWQ, you will need Colab Pro).**

Model = "TheBloke/Mythalion-13B-AWQ" #@param ["TheBloke/Mythalion-13B-AWQ", "TheBloke/MythoMax-L2-13B-AWQ", "TheBloke/Pygmalion-2-13B-AWQ", "TheBloke/MLewd-L2-Chat-13B-AWQ"]{allow-input: true}
#@markdown **The quantization method to use. Set to `None` if your model is not quantized.**
Quantization = "awq" #@param ["gptq", "awq", "None"]
#@markdown **Adjust this and the Context Length slider if you're running into COOM (CUDA Out Of Memory) issues!**
GPU_Memory_Utilization = 0.88 #@param {type:"slider", min:0, max:1, step:0.01}
#@markdown **The free Colab GPU may not have enough memory to accomodate more than 8192 Context Length for most models.**
Context_Length = 4096 #@param {type:"slider", min:1024, max:32768, step:1024}
#@markdown **Leave this as default unless you've paid for Colab Pro and are using a more powerful GPU. Note: Only affects non-quantized models.**
DataType = "float16" #@param ["float16", "bfloat16", "float32"]
#@markdown **The API type to use. We currently support OpenAI and KoboldAI.**
API_Type = "KoboldAI" #@param ["KoboldAI", "OpenAI"]
#@markdown **Enter an API key if you've selected the OpenAI API type.**
OpenAI_API_Key = "" #@param []{allow-input: true}


%pip show aphrodite-engine &> /dev/null && echo "Existing Aphrodite Engine installation found. Updating..." && pip uninstall aphrodite-engine -q -y
!echo "Installing the Aphrodite Engine, this will take a while..."
%pip install aphrodite-engine==0.4.2 > /dev/null 2>&1
!echo "Installation successful! Starting the engine now."


!wget -q -c https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!echo "Creating a Cloudflare URL..."
!nohup ./cloudflared-linux-amd64 tunnel --url http://127.0.0.1:2242 &> nohup.out &
!sleep 10
!echo "============================================================"
!echo "Please copy this URL:"
!grep -o 'https://[^ ]*.trycloudflare.com' nohup.out
!echo "============================================================"

model = Model
data_type = DataType
gpu_memory_utilization = GPU_Memory_Utilization
context_length = Context_Length
api_type = API_Type
api_key = OpenAI_API_Key

if api_type == "KoboldAI":
  if Quantization == "None":
    !python -m aphrodite.endpoints.kobold.api_server --model $model --dtype $data_type --host 127.0.0.1 --gpu-memory-utilization $gpu_memory_utilization --max-model-len $context_length --max-log-len 0 &
  else:
    !python -m aphrodite.endpoints.kobold.api_server --model $model --dtype float16 --host 127.0.0.1 -q $Quantization --gpu-memory-utilization $gpu_memory_utilization --max-model-len $context_length --max-log-len 0 &
elif api_type == "OpenAI":
  if Quantization == "None":
    !python -m aphrodite.endpoints.openai.api_server --model $model --dtype $data_type --host 127.0.0.1 --gpu-memory-utilization $gpu_memory_utilization --max-model-len $context_length --max-log-len 0 --api-keys $api_key &
  else:
    !python -m aphrodite.endpoints.openai.api_server --model $model --dtype float16 --host 127.0.0.1 -q $Quantization --gpu-memory-utilization $gpu_memory_utilization --max-model-len $context_length --max-log-len 0 --api-keys $api_key &

